In [1]:
# !pip3 install gspread
# !pip3 install --upgrade google-api-python-client oauth2client 

In [2]:
import requests 
import json
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [39]:
def date_to_url(s):
    """ input 17/05/1996 
        return 19962F052F17 (url format)"""
    l = s.split('/')
    if len(l)==2:
        year=2022
        day,month = l
    elif len(l)==3:
        day,month,year = l

    return f'{year}%2F{month}%2F{day}'
    

In [40]:
gc = gspread.service_account() #gc = gspread.service_account(filename='path/to/the/downloaded/file.json')
sh = gc.open("is-there-tgvmax")


In [41]:
worksheet_list = [sh for sh in sh.worksheets() if '@' in sh.title]
sh=worksheet_list[0]

'FALSE'

In [57]:
origine,destination,date,heure_min,heure_max,_,email_envoye,force_update=sh.row_values(2)


In [74]:
print(origine,destination,date,heure_min,heure_max,_,email_envoye,force_update)

PARIS+(intramuros) GRENOBLE 22/06 7 23  FALSE FALSE


In [59]:
request_start = 'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card'
full_request = f'{request_start}&refine.destination={destination}&refine.origine={origine}&refine.date={date_to_url(date)}'
r=requests.get(full_request)

In [60]:
full_request

'https://ressources.data.sncf.com/api/records/1.0/search/?dataset=tgvmax&q=&sort=-date&facet=date&facet=origine&facet=destination&facet=od_happy_card&refine.destination=GRENOBLE&refine.origine=PARIS+(intramuros)&refine.date=2022%2F06%2F22'

In [61]:
r.text

'{"nhits": 6, "parameters": {"dataset": ["tgvmax"], "rows": 10, "start": 0, "sort": ["-date"], "facet": ["date", "origine", "destination", "od_happy_card"], "format": "json", "timezone": "UTC"}, "records": [{"datasetid": "tgvmax", "recordid": "6400fcb4cee7348e9beeebc234e746b0ca6f924b", "fields": {"heure_arrivee": "13:42", "code_equip": "TGF", "date": "2022-06-22", "train_no": "6311", "destination": "GRENOBLE", "origine": "PARIS (intramuros)", "entity": "PAGRENOBLE", "origine_iata": "FRPLY", "od_happy_card": "NON", "axe": "SUD-EST", "destination_iata": "FRGNB", "heure_depart": "9:53"}, "record_timestamp": "2022-06-14T10:00:01.512Z"}, {"datasetid": "tgvmax", "recordid": "b8cfbff3eb0885a7a783ecb293b5b4490c039900", "fields": {"heure_arrivee": "10:48", "code_equip": "TGF", "date": "2022-06-22", "train_no": "6905", "destination": "GRENOBLE", "origine": "PARIS (intramuros)", "entity": "PAGRENOBLE", "origine_iata": "FRPLY", "od_happy_card": "NON", "axe": "SUD-EST", "destination_iata": "FRGNB",

In [65]:
dico = json.loads(r.text)

In [79]:
def time_to_int(date):
    return int(date.split(':')[0])

In [87]:
for voyage in dico['records'] :
    heure_depart = time_to_int(voyage['fields']['heure_depart'])
    if heure_depart>=int(heure_min) and heure_depart<=int(heure_max):
        print('GO')

GO
GO
GO
GO
GO
GO


In [67]:
dico

{'nhits': 6,
 'parameters': {'dataset': ['tgvmax'],
  'rows': 10,
  'start': 0,
  'sort': ['-date'],
  'facet': ['date', 'origine', 'destination', 'od_happy_card'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'tgvmax',
   'recordid': '6400fcb4cee7348e9beeebc234e746b0ca6f924b',
   'fields': {'heure_arrivee': '13:42',
    'code_equip': 'TGF',
    'date': '2022-06-22',
    'train_no': '6311',
    'destination': 'GRENOBLE',
    'origine': 'PARIS (intramuros)',
    'entity': 'PAGRENOBLE',
    'origine_iata': 'FRPLY',
    'od_happy_card': 'NON',
    'axe': 'SUD-EST',
    'destination_iata': 'FRGNB',
    'heure_depart': '9:53'},
   'record_timestamp': '2022-06-14T10:00:01.512Z'},
  {'datasetid': 'tgvmax',
   'recordid': 'b8cfbff3eb0885a7a783ecb293b5b4490c039900',
   'fields': {'heure_arrivee': '10:48',
    'code_equip': 'TGF',
    'date': '2022-06-22',
    'train_no': '6905',
    'destination': 'GRENOBLE',
    'origine': 'PARIS (intramuros)',
    'entity': 'PAGRENOBLE